In [168]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [169]:
import os
import sqlite3
import pandas as pd
import requests
from typing import Optional
API_KEY = str(os.getenv("LASTFM_KEY"))
API_SECRET = str(os.getenv("LASTFM_SECRET"))
headers = {
    'user-agent': "S&DS 230"
}

In [170]:
from tqdm.auto import tqdm
tqdm.pandas()

In [171]:
SQL_FILEPATH = os.getenv("PLAYBACK_FILE")
lyrics_db = sqlite3.connect(SQL_FILEPATH)
df = pd.read_sql_query("SELECT * FROM tracks", lyrics_db)

DatabaseError: Execution failed on sql 'SELECT * FROM tracks': no such table: tracks

In [ ]:
from difflib import SequenceMatcher

# Modified from: https://stackoverflow.com/a/17388505
def match(orig_name, found_name):
    return SequenceMatcher(None, orig_name, found_name).ratio()

In [ ]:
match("B.o.B & Hayley Williams", "B.o.B")

In [173]:
# Improve: search for tracks where last.fm artist is similar to the searched artist but has more listener count
def find_track(track: str, artist: Optional[str] = None):
    MIN_LISTEN = 100
    payload = {
        'api_key': API_KEY,
        'method': 'track.search',
        'format': 'json',
        'track': track,
    }
    no_artist = query_lastfm(payload)
    w_artist = None
    # print(no_artist)
    if artist and no_artist:
        payloadArtist = payload
        payloadArtist["artist"] = artist
        w_artist = query_lastfm(payloadArtist)
    return_payload = None
    if no_artist and w_artist:
        if w_artist["last_fm_listeners"] > MIN_LISTEN and w_artist["last_fm_listeners"] * 100 > no_artist["last_fm_listeners"] :
            return_payload = w_artist
        elif no_artist["last_fm_listeners"] > MIN_LISTEN and match(no_artist["artist"], artist) > .25:
            return_payload = no_artist
    elif w_artist and w_artist["last_fm_listeners"] > MIN_LISTEN:
        return_payload = w_artist
    elif no_artist and no_artist["last_fm_listeners"] > MIN_LISTEN and match(no_artist["artist"], artist) > .25:
        return_payload = no_artist
    # print("RETURN PAYLOAD")
    # print(return_payload)
    return return_payload
    # B.o.B & Hayley Williams

In [174]:
def query_lastfm(payload):
    try:
        r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
        if r.status_code != 200:
            return None
        num_results = int(r.json()["results"]["opensearch:totalResults"])
        if num_results > 0:
            matches = r.json()["results"]["trackmatches"]["track"][0]["url"]
            listeners = int(r.json()["results"]["trackmatches"]["track"][0]["listeners"])
            artist = r.json()["results"]["trackmatches"]["track"][0]["artist"]
            return {"last_fm_url": matches, "last_fm_listeners": listeners, "artist": artist}
        else:
            return None
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        return None

In [175]:
def find_track_series(series):
    track_data = find_track(series["track"], series["artist"])
    if track_data:
        series["last_fm_url"] = track_data["last_fm_url"]
        series["last_fm_listeners"] = track_data["last_fm_listeners"]
    else:
        series["last_fm_url"] = None
        series["last_fm_listeners"] = None
    return series

In [176]:
df = df.progress_apply(find_track_series, axis=1)

  0%|          | 0/3595 [00:00<?, ?it/s]

In [181]:
df.drop("level_0", axis=1).drop("index", axis=1).to_sql("tracks", lyrics_db, if_exists="replace")

3595

In [ ]:
find_track("Show Me the Meaning of Being Lonely", "The Backstreet Boys")

,track_id,year,artist,track,rank,link,lyrics,artist_id,artist_appearances,last_fm_url,last_fm_listeners
0,0,1985,USA For Africa,We Are the World,1,/charts/top-100-songs/video/1985/USA-For-Afric...,We Are the World Lyrics\nThere comes a time wh...,1,1,https://www.last.fm/music/USA+for+Africa/_/We+...,83127.0
1,1,1985,A-Ha,Take On Me,2,/charts/top-100-songs/video/1985/AHa-Take-On-Me,None,2,4,https://www.last.fm/music/a-ha/_/Take+on+Me,1545502.0
2,2,1985,Foreigner,I Want to Know What Love Is,3,/charts/top-100-songs/video/1985/Foreigner-I-W...,I Want to Know What Love Is Lyrics\nI've gotta...,3,1,https://www.last.fm/music/Foreigner/_/I+Want+t...,469941.0
3,3,1985,Tears For Fears,Shout,4,/charts/top-100-songs/video/1985/Tears-For-Fea...,"Shout Lyrics\nShout, shout, let it all out\nTh...",4,5,https://www.last.fm/music/Tears+for+Fears/_/Shout,662279.0
4,4,1985,Lionel Richie,"Say You, Say Me",5,/charts/top-100-songs/video/1985/Lionel-Richie...,"Say You, Say Me Lyrics\nSay you, say me\nSay i...",5,3,https://www.last.fm/music/Lionel+Richie/_/Say+...,232323.0
...,...,...,...,...,...,...,...,...,...,...,...
3590,3590,2021,Marshmello & Jonas Brothers,Leave Before You Love Me,96,/charts/top-100-songs/video/2021/Marshmello--J...,Leave Before You Love Me Lyrics\nI see you cal...,1808,1,https://www.last.fm/music/Marshmello+&+Jonas+B...,4989.0
3591,3591,2021,Maneskin,Beggin,97,/charts/top-100-songs/video/2021/Maneskin-Beggin,"Beggin’ Lyrics\nPut your loving hand out, baby...",1809,1,https://www.last.fm/music/M%C3%A5neskin/_/Begg...,440454.0
3592,3592,2021,Chris Young + Kane Brown,Famous Friends,98,/charts/top-100-songs/video/2021/Chris-Young--...,Famous Friends Lyrics\nMy buddy Brandon holds ...,1810,1,https://www.last.fm/music/Chris+Young/_/Famous...,17698.0
3593,3593,2021,Nelly & Florida Georgia Line,Lil Bit,99,/charts/top-100-songs/video/2021/Nelly--Florid...,"Lil Bit Lyrics\nWhat up, Nelly?\nYou ready to ...",1811,1,None,NaN
